## Landsat-7 cloud/shadow-free composite

This example aims to create a Landsat-7 cloud/shadow free composite on, or as close as possible to 22-23 November 2016.  The area of interest covers a range of natural, agricultural and urban areas around Stellenbosch, South Africa.

### Setup

In [1]:
# Uncomment if you don't have geedim and geemap installed
# !pip install geedim geemap
# !pip install geedim geedim


In [2]:
import ee
import geedim as gd
import geemap.foliumap as geemap

# initialise earth engine with the high-volume endpoint
gd.Initialize()


### Create and search a Landsat-7 collection

In [3]:
# geojson search region
region = {
    'type': 'Polygon', 'coordinates': [[
        (19.075, -34.115), (19.075, -33.731), (18.723, -33.731), 
        (18.723, -34.115), (19.075, -34.115)
    ]]
}

# create and search the Landsat-7 collection
coll = gd.MaskedCollection.from_name('LANDSAT/LE07/C02/T1_L2')
filt_coll = coll.search('2016-10-22', '2016-12-22', region, cloudless_portion=40)

# print the search results
print(filt_coll.schema_table)
print(filt_coll.properties_table)

ABBREV    DESCRIPTION
--------- ---------------------------------------
ID        Earth Engine image id
DATE      Image capture date/time (UTC)
FILL      Portion of valid pixels (%)
CLOUDLESS Portion of cloud/shadow free pixels (%)
GRMSE     Orthorectification RMSE (m)
SAA       Solar azimuth angle (deg)
SEA       Solar elevation angle (deg)
ID                                          DATE              FILL CLOUDLESS GRMSE   SAA   SEA
------------------------------------------- ---------------- ----- --------- ----- ----- -----
LANDSAT/LE07/C02/T1_L2/LE07_175083_20161031 2016-10-31 08:37 65.52     59.11  4.78 59.52 58.68
LANDSAT/LE07/C02/T1_L2/LE07_175084_20161031 2016-10-31 08:38 66.34     44.26  6.44 57.92 57.72
LANDSAT/LE07/C02/T1_L2/LE07_175083_20161116 2016-11-16 08:37 65.43     52.95  4.89 67.06 61.31
LANDSAT/LE07/C02/T1_L2/LE07_175084_20161116 2016-11-16 08:38 67.79     49.11  6.74 65.06 60.50
LANDSAT/LE07/C02/T1_L2/LE07_175083_20161202 2016-12-02 08:37 65.36     65.36  4.90 73.

#### Notes on search results

* The 2016-11-16 images are closest to the target dates, but have some cloud in them (CLOUDLESS < FILL)
* No single image has full coverage (FILL=100) of the search area.  In part, this is just due to the 
footprint of the images, but is also a result of the 
[Landsat-7 SLC failure](<https://en.wikipedia.org/wiki/Landsat_7#Scan_Line_Corrector_failure>).

### Find composite images

Here, we find *mosaic*, and *q-mosaic* composite images, prioritising images closest to 2016-11-22 by specifying the `date` parameter.

In [4]:
mosaic_im = filt_coll.composite(method=gd.CompositeMethod.mosaic, date='2016-11-22')
q_mosaic_im = filt_coll.composite(method=gd.CompositeMethod.q_mosaic, date='2016-11-22')

### Visualise search result and composite images

In [5]:
# visualise the search result and composite images

l7_vis_params = dict(min=7300, max=13000, bands=['SR_B3', 'SR_B2', 'SR_B1'], gamma=1.5)
map = geemap.Map()

map.centerObject(ee.Geometry(region), 12)
for im_id in filt_coll.properties.keys():
    im = gd.MaskedImage.from_id(im_id, mask=False)
    map.addLayer(im.ee_image.clip(region), l7_vis_params, im_id[-20:])

map.addLayer(mosaic_im.ee_image.clip(region), l7_vis_params, 'Mosaic composite')
map.addLayer(q_mosaic_im.ee_image.clip(region), l7_vis_params, 'Q-mosaic composite')

region_im = ee.Image().byte().paint(featureCollection=ee.Geometry(region), width=2, color=1)
map.addLayer(region_im, dict(palette=['FF0000']), 'Region')

map


#### Visualisation notes

* The *mosaic* method image contains some artefacts due to remnant cloud in the masked input images.
* By prioritising pixels with the highest distance to cloud, the *q-mosaic* method is robust to imperfect 
cloud/shadow masking, and produces a composite free of cloud artefacts.

### Download

Lastly, we download the *q-mosaic* composite.  `crs`, `scale` and `region` parameters must be specified, as the composite has no fixed projection.

In [6]:
# download the q_mosaic composite image, specifying crs, scale and region as they have no fixed projection
q_mosaic_im.download('l7_q_mosaic_im.tif', crs='EPSG:3857', scale=30, region=region, dtype='uint16', overwrite=True)

l7_q_mosaic_im.tif: |█████████▉| 108M/108M (raw) [100.0%] in 00:34 (eta: 00:00)    
